In [1]:
import os
import boto3
from botocore.config import Config
from dotenv import load_dotenv
import pandas as pd
import sqlite3
from sspipe import p, px
load_dotenv('env')

True

Download the raw files to local directory

In [2]:
folder_name = 'raw_data'

def s3_client():
    config = Config(s3={"use_accelerate_endpoint": True, "addressing_style": "path"})
    s3_resource = boto3.resource("s3", config=config,
                     aws_access_key_id = os.getenv('aws_access_key_id'),             
                     aws_secret_access_key = os.getenv('aws_secret_access_key'),
                     region_name = os.getenv('region_name'))
    
    return s3_resource.meta.client
    
s3_client = s3_client()

# create directory 'raw_data' if it does not exist yet
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
# get names of all files in s3fld/599Team12/raw_data/ if size > 0
objects = s3_client.list_objects(Bucket='s3fld', Prefix=f'599Team12/{folder_name}')['Contents']
file_names = [obj['Key'] for obj in objects if obj['Size'] > 0]

# download each file to local directory
for file_name in file_names:   
    if not os.path.exists(folder_name + '/' + file_name.split('/')[-1]):
        s3_client.download_file('s3fld', file_name,  folder_name + '/' + file_name.split('/')[-1])

The code below takes a `folder_name` and `database_name` as input. It then reads all the files in the specified folder and creates a Pandas DataFrame for each file. The DataFrame will be stored in a `sqlite database`. For example, if `folder_name` contains a file named `foo.csv`, the database will have a table with name `foo`.

In [3]:
def get_database(folder_name='raw_data', database_name='database.db'):
    if os.path.exists(database_name):
        return sqlite3.connect(database_name)
    
    db = sqlite3.connect(database_name)

    for file in os.listdir(folder_name):
        name = file.split('.')[0]
        print(f'Reading `{name}` dataframe')
        data = pd.read_csv(f'{folder_name}/{file}', encoding='utf-8')
        data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
        data.to_sql(name, db, if_exists='replace', index=False)  
    
    db.commit()
    return db

db = get_database()

Below will ensure that tables are created and the database can be queried successfully.

In [4]:
sample_query = """
    SELECT 
        `review`.`date`, 
        `review`.`stars`, 
        `review`.`text`,
        `business`.`name` AS `business_name`,
        `business`.`city`
            
    FROM `review` 
    
    JOIN `business` ON 
        `review`.`business_id` = `business`.`business_id`
            
    LIMIT 5
    """

pd.read_sql_query(sample_query, db)

,date,stars,text,business_name,city
0,2018-07-07 22:09:11,3,"If you decide to eat here, just be aware it is...",Turning Point of North Wales,North Wales
1,2012-01-03 15:28:18,5,I've taken a lot of spin classes over the year...,Body Cycle Spinning Studio,Philadelphia
2,2014-02-05 20:30:30,3,Family diner. Had the buffet. Eclectic assortm...,Kettle Restaurant,Tucson
3,2015-01-04 00:01:03,5,"Wow! Yummy, different, delicious. Our favo...",Zaika,Philadelphia
4,2017-01-14 20:54:15,4,Cute interior and owner (?) gave us tour of up...,Melt,New Orleans


Query the data that will be tokinized

In [5]:
sql_business_category = 'SELECT `business_id`, `name`, `categories` FROM `business`'
sql_reviews_business_id = 'SELECT `business_id`, `review_id`, `date`, `stars`,`text` FROM `review`'

restaurants = (pd.read_sql(sql_business_category, db)
               | px.query("categories.str.contains('Restaurants') == True"))

reviews_for_resturants = (pd.read_sql(sql_reviews_business_id, db)
                          | px[px.business_id.isin(restaurants.business_id)]
                          | px.join(restaurants.set_index('business_id'), on='business_id')
                          | px[['business_id', 'review_id' , 'date', 'stars', 'text', 'name' ]]
)

print(f'There are {len(reviews_for_resturants):,} numebr of reviews for restaurants')
reviews_for_resturants.sample(5)

There are 4,724,471 numebr of reviews for restaurants


,review_id,date,stars,text,name
647276,FOWMAUp12zVaf-GddCpUUQ,2019-06-27 22:02:44,5,What can I say about a shop that I've loved ev...,The Blind Tiger Cafe - Ybor City
809507,eBuHOKkx6H9P-zAwuDSAwg,2014-01-21 02:05:43,2,It was kind of odd. It's obviously a fast food...,Steak ’n Shake
6016548,yre5g393rn2FBdcbdFL_9Q,2019-10-12 01:01:43,2,We've been doing business with Chris's for sev...,Chris' Pizza Village
6298942,RBLOdFQEVUhgL52vwJuOxw,2018-02-02 01:38:53,2,Overpriced and bland food. A meat and three sh...,Bishop's Meat and Three
3655034,tKRlCpQRAYMAySNIdJDOUw,2018-06-01 21:48:55,5,"Really friendly and attentive bartenders, grea...",Bridge Tap House and Wine Bar


save the related data to a table for easy access

In [6]:
reviews_for_resturants.to_sql('resturants_review', db, if_exists='replace', index=False)
db.close()